In [61]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# from keras.models import load_model

In [2]:
predicoes_cnn = np.array([])
predicoes_svm = np.array([])
predicoes_real= np.array([])

f_path = 'predicoes/'

n = 1404
# Leitura dos resultados salvos em arquivo

# CNN
for i in range(5):
    print(f'Leitura do arquivo:', f'{f_path}predicoes_cnn_fold{i+1}.txt')
    arq_cnn = open(f'{f_path}predicoes_cnn_fold{i+1}.txt')
    arquivo_completo = arq_cnn.read().split('\n')
    for  j in range(len(arquivo_completo)-1):
        aux = arquivo_completo[j].split(';')
        vetor_aux = np.array([])
        for k in aux:
            if k != '':
                vetor_aux = np.append(vetor_aux, float(k))
        if j == 0 and i == 0:
            predicoes_cnn = np.append(predicoes_cnn, vetor_aux)
        else:
            predicoes_cnn = np.vstack((predicoes_cnn, vetor_aux))
    print(f'Fim da leitura do arquivo:', f'{f_path}predicoes_cnn_fold{i+1}.txt')
    arq_cnn.close()

print()
# SVM
for i in range(5):
    print(f'Leitura do arquivo:', f'{f_path}predicoes_svm_fold{i+1}.txt')
    arq = open(f'{f_path}predicoes_svm_fold{i+1}.txt')
    arquivo_completo = arq.read().split('\n')
    for  j in range(len(arquivo_completo)-1):
        aux = arquivo_completo[j].split(';')
        vetor_aux = np.array([])
        for k in aux:
            if k != '':
                vetor_aux = np.append(vetor_aux, float(k))
        if j == 0 and i == 0:
            predicoes_svm = np.append(predicoes_svm, vetor_aux)
        else:
            predicoes_svm = np.vstack((predicoes_svm, vetor_aux))
    print(f'Fim da leitura do arquivo:', f'{f_path}predicoes_svm_fold{i+1}.txt')
    arq.close()

print()
# Real
for i in range(5):
    print(f'Leitura do arquivo:', f'{f_path}predicoes_real_fold{i+1}.txt')
    arq = open(f'{f_path}predicoes_real_fold{i+1}.txt')
    arquivo_completo = arq.read().split('\n')
    for  j in range(len(arquivo_completo)-1):
        aux = arquivo_completo[j].split(';')
        vetor_aux = np.array([])
        for k in aux:
            if k != '':
                vetor_aux = np.append(vetor_aux, float(k))
        if j == 0 and i == 0:
            predicoes_real = np.append(predicoes_real, vetor_aux)
        else:
            predicoes_real = np.vstack((predicoes_real, vetor_aux))
    print(f'Fim da leitura do arquivo:', f'{f_path}predicoes_real_fold{i+1}.txt')
    arq.close()

Leitura do arquivo: predicoes/predicoes_cnn_fold1.txt
Fim da leitura do arquivo: predicoes/predicoes_cnn_fold1.txt
Leitura do arquivo: predicoes/predicoes_cnn_fold2.txt
Fim da leitura do arquivo: predicoes/predicoes_cnn_fold2.txt
Leitura do arquivo: predicoes/predicoes_cnn_fold3.txt
Fim da leitura do arquivo: predicoes/predicoes_cnn_fold3.txt
Leitura do arquivo: predicoes/predicoes_cnn_fold4.txt
Fim da leitura do arquivo: predicoes/predicoes_cnn_fold4.txt
Leitura do arquivo: predicoes/predicoes_cnn_fold5.txt
Fim da leitura do arquivo: predicoes/predicoes_cnn_fold5.txt

Leitura do arquivo: predicoes/predicoes_svm_fold1.txt
Fim da leitura do arquivo: predicoes/predicoes_svm_fold1.txt
Leitura do arquivo: predicoes/predicoes_svm_fold2.txt
Fim da leitura do arquivo: predicoes/predicoes_svm_fold2.txt
Leitura do arquivo: predicoes/predicoes_svm_fold3.txt
Fim da leitura do arquivo: predicoes/predicoes_svm_fold3.txt
Leitura do arquivo: predicoes/predicoes_svm_fold4.txt
Fim da leitura do arquivo

In [3]:
print(np.shape(predicoes_cnn))
print(np.shape(predicoes_svm))
print(np.shape(predicoes_real))

(7020, 15)
(7020, 15)
(7020, 1)


In [66]:
def borda_count(svm_preds, cnn_preds):
    # Calcular as bordas para cada classe em cada amostra
    bordas = np.abs(svm_preds - cnn_preds)

    # Determinar a previsão final pela classe com maior borda
    previsoes_finais = np.argmax(bordas, axis=1)

    return previsoes_finais

def soma_simples(svm_preds, cnn_preds):
    preds_compinadas = svm_preds + cnn_preds

    return np.argmax(preds_compinadas, axis=1)


def media_simples(svm_preds, cnn_preds):
    # Calcule as previsões combinadas por média simples
    preds_combinadas = (svm_preds + cnn_preds) / 2.0

    # Obtenha as classes preditas usando o argmax
    classes_preditas = np.argmax(preds_combinadas, axis=1)

    return classes_preditas


def media_ponderada(svm_preds, cnn_preds, peso_svm, peso_cnn):
    # Calcule as previsões combinadas por média ponderada
    preds_combinadas = (peso_svm * svm_preds + peso_cnn *
                        cnn_preds) / (peso_svm + peso_cnn)

    # Obtenha as classes preditas usando o argmax
    classes_preditas = np.argmax(preds_combinadas, axis=1)

    return classes_preditas


def maiores_valores(svm_preds, cnn_preds):
    # Realize a fusão selecionando os maiores valores entre as previsões de cada modelo
    fusion_preds = np.maximum(svm_preds, cnn_preds)

    # Obtenha as classes preditas usando o argmax
    classes_preditas = np.argmax(fusion_preds, axis=1)

    return classes_preditas

def maioria_ponderada(svm_preds, cnn_preds, peso_svm, peso_cnn):
    fusion_preds = peso_svm * svm_preds + peso_cnn * cnn_preds

    classes_preditas = np.argmax(fusion_preds, axis=1)

    return classes_preditas

def fusao_com_rejeicao(svm_preds, cnn_preds, limiar_rejeicao=0.4):
    votos_totais = svm_preds + cnn_preds
    previsoes_finais = np.argmax(votos_totais, axis=1)
    confianca = np.max(votos_totais, axis=1)
    previsoes_finais[confianca < limiar_rejeicao] = -1  # Marcando amostras não confiáveis com -1
    return previsoes_finais

def selecao_de_modelo(svm_preds, cnn_preds):
    return np.argmax(svm_preds, axis=1)

# def stacking(svm_preds, cnn_preds, y_true):
#     X = np.column_stack((svm_preds, cnn_preds))
#     X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

#     with open('./saida2/modelo_svm.pkl', 'rb') as arquivo_svm:
#         svm_model = pickle.load(arquivo_svm)
#     cnn_model = load_model('./saida2/model.h5')
    
#     svm_model.fit(X_train, y_train)
#     cnn_model.fit(X_train, y_train)

#     svm_preds_val = svm_model.predict(X_val)
#     cnn_preds_val = cnn_model.predict(X_val)

#     meta_X_val = np.column_stack((svm_preds_val, cnn_preds_val))

#     meta_model = RandomForestClassifier()
#     meta_model.fit(meta_X_val, y_val)

#     return meta_model.predict(np.column_stack((svm_preds, cnn_preds)))

# def blending(svm_preds, cnn_preds, y_true, blend_ratio=0.7):
#     X = np.column_stack((svm_preds, cnn_preds))
#     X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

#     svm_model = svm.SVC()
#     cnn_model = Your_CNN_Model()
    
#     svm_model.fit(X_train, y_train)
#     cnn_model.fit(X_train, y_train)

#     svm_preds_val = svm_model.predict(X_val)
#     cnn_preds_val = cnn_model.predict(X_val)

#     # Blending
#     previsoes_finais = blend_ratio * svm_preds_val + (1 - blend_ratio) * cnn_preds_val
#     previsoes_finais = np.argmax(previsoes_finais, axis=1)

#     return previsoes_finais

# terminar!!!
def votacao_hierarquica(svm_preds, cnn_preds, peso_svm, peso_cnn):
    # Camada 1
    votos_maioria_ponderada = peso_svm * svm_preds + peso_cnn * cnn_preds

    # Camada 2
    diferenca_votos = np.abs(svm_preds - cnn_preds)


    camada_1 = (svm_preds + cnn_preds) / 2
    camada_2 = np.argmax(camada_1, axis=1)
    camada_3 = camada_2  # Aqui pode haver uma terceira camada e assim por diante, se necessário
    return camada_3

In [67]:
nomes_metodos = ['Votação por Borda', 'Soma Simples', 'Média Simples',
                 'Média Ponderada', 'Maiores Valores', 'Maioria Ponderada', 'Fusao com rejeicao', 'Selecao de modelo', 'Votacao hierarquica', 'Stacking', 'Blending', 'Votacao hierarquica']

lista_acuracia = np.array([])

preds_teste = borda_count(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_borda_count:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = soma_simples(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_soma_simples:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)


preds_teste = media_simples(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_media_simples:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = media_ponderada(predicoes_svm, predicoes_cnn, 2, 8)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_media_ponderada:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = maiores_valores(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_maiores_valores:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = maioria_ponderada(predicoes_svm, predicoes_cnn, 1, 2)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_maioria_ponderada:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = fusao_com_rejeicao(predicoes_svm, predicoes_cnn, limiar_rejeicao=0.8)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_fusao_com_rejeicao:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = selecao_de_modelo(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_selecao_de_modelo:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = votacao_hierarquica(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_votacao_hierarquica:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

# preds_teste = stacking(predicoes_svm, predicoes_cnn, predicoes_real)
# acc_aux = accuracy_score(predicoes_real, preds_teste)
# print("acc_stacking:", acc_aux)
# lista_acuracia = np.append(lista_acuracia, acc_aux)

# preds_teste = blending(predicoes_svm, predicoes_cnn, y_true, blend_ratio=0.6)
# acc_aux = accuracy_score(predicoes_real, preds_teste)
# print("acc_stacking:", acc_aux)
# lista_acuracia = np.append(lista_acuracia, acc_aux)


melhor_metodo = np.argmax(lista_acuracia)
melhor_acuracia = lista_acuracia[melhor_metodo]
nome_melhor_metodo = nomes_metodos[melhor_metodo]

print(f'O melhor método é o {nome_melhor_metodo} com acuracia de {melhor_acuracia}')

acc_borda_count: 0.881908831908832
acc_soma_simples: 0.9185185185185185
acc_media_simples: 0.9185185185185185
acc_media_ponderada: 0.9173789173789174
acc_maiores_valores: 0.9146723646723647
acc_maioria_ponderada: 0.9195156695156695
acc_fusao_com_rejeicao: 0.9032763532763532
acc_selecao_de_modelo: 0.6417378917378918
acc_stacking: 0.9185185185185185
O melhor método é o Maioria Ponderada com acuracia de 0.9195156695156695
